In [2]:
import numpy as np
import pandas as pd

In [ ]:
# SCENARIO: murder mystery on spaceship (I liked the idea, and I wanted to have a more complicated version,
# where some suspects fully can't use certain weapons)
# There are 6 weapons and 5 possible suspects
# Each suspect has a different likelihood function describing the conditional probability of them using a given weapon

weapons = np.array([
    'raygun',
    'plasma-torch',
    'brain-hack',
    'teleporter-hack',
    'parasite-beetle',
    'artifact'
])

suspects = np.array([
    'captain',
    'engineer',
    'pilot',
    'prisoner',
    'computer',
])

likelihoods = {
    # Captain:
    # - Very good shot with a raygun and keep theirs on their person
    # - Captain has access to every room on the ship so could obtain a plasma torch pretty easily
    # - Bad with tech, so brain interface hack or sabotaging the teleporter is really unlikely
    # - Has a known immunity to the parasite beetle's saliva, so it would be a pretty convenient
    #       murder weapon for them to use, but less likely than the gun
    # - They have no idea how to use the artifact, fully 0% of killing a person with it
    "captain":{
        "raygun":0.69,
        "plasma-torch":0.21,
        "brain-hack":0.01,
        "teleporter-hack":0.01,
        "parasite-beetle":0.08,
        "artifact":0
    },
    # Engineer:
    # - They've never used a gun before, they'd probably decide it's too risky to use as a murder weapon
    # - Definitely they own a plasma torch, so if they're killing someone it's probably with that
    # - brain interface hacking isn't their expertise, but they could hack the teleporter easily.
    # - Insects are sacred in their culture, so they'd never use one as a murder weapon (unless they're
    #       being clever and are using that as an alibi)
    # - They've been studying the artifact and have unlocked some of its abilities, some chance that they could kill someone with it
    "engineer" : {
        "raygun":0.03,
        "plasma-torch":0.6,
        "brain-hack":0.05,
        "teleporter-hack":0.24,
        "parasite-beetle":0.03,
        "artifact":0.05
    },
    # Pilot:
    # - The pilot goes with the captain on field missions, probably good with a raygun,
    #       but a plasma torch is probably easier for them to acquire
    # - They use a more advanced brain-machine interface to pilot the ship with their mind,
    #       it might be the easiest way to kill someone
    # - The ship's pilot can override other teleportation commands, but that means anything sketchy they do is recorded
    #       so they'd be unlikely to kill someone with a teleporter hack.
    # - Insectophobe/germophobe from being raised on a space station, not going to use a parasite beetle
    # - It's possible they threatened the engineer into figuring out how to kill someone with the artifact? otherwise unlikely
    "pilot":{
        "raygun":0.3,
        "plasma-torch":0.3,
        "brain-hack":0.3,
        "teleporter-hack":0.02,
        "parasite-beetle":0.02,
        "artifact":0.06
    },
    # Prisoner:
    # - They've never used or seen a human weapon before, so unlikely to even register raygun as a weapon
    # - Plasma torches produce ultrasound noise that hurt the prisoner's ears, unlikely to use one (unless they used it to break out of prison)
    # - The prisoner has psychic powers equivalent to a brain interface hack
    # - Teleporters were developed on the prisoner's homeworld, very familiar with how to sabotage them
    # - Parasite beetles are useful for killing at a distance, so would be possible to kill from inside their cell (they still need to break
    #        out to acquire the beetle though)
    # - Artifact is also from their homeworld, but out of respect for the consciousness inside the artifact they probably wouldn't make it kill someone
    "prisoner":{
        "raygun":0.02,
        "plasma-torch":0.03,
        "brain-hack":0.27,
        "teleporter-hack":0.35,
        "parasite-beetle":0.28,
        "artifact":0.05
    },
    # Computer:
    # - Can't interact with physical objects (raygun, plasma torch, or parasite beetle)
    # - The computer manages the brain interface network, and would 
    # - The computer has less direct control over the teleporter and would be less inclined to kill with a teleporter hack
    # - The computer has secretly been studying the artifact and could kill a person with it.
    "computer":{
        "raygun":0.0,
        "plasma-torch":0.0,
        "brain-hack":0.5,
        "teleporter-hack":0.05,
        "parasite-beetle":0.0,
        "artifact":0.45
    },

}

prior_probs = np.array([
    0.12,   # captain is hotheaded but probably wouldn't kill a crew member
    0.02,  # engineer is very level-headed and is a pacifist, very unlikely to be them
    0.28,  # pilot is has an antisocial streak and could get angry enough to kill someone
    0.38,  # prisoner resents being held captive. There's a low change of them actually escaping, but nearly 100% change of them killing someone
    0.2   # computer is an experimental model and may have suffered corruption from interfacing with artifact
])

# changes the nested dictionaries into a 2D array for vectorized access
likelihoods_arr=np.array([
    [likelihoods[s][w] for w in weapons]
    for s in suspects
])

In [18]:
def draw_prior(suspects, probs, num_sims):
    """
    Draws a number of samples from the prior distribution of suspects on the ship.
    To simplify later steps, this function returns only the indices of the suspects.

    Parameters
    ----------
    suspects : array-like, shape (N,)
            The array of suspects to draw from
    probs : array-like, shape (N,)
            The prior probabilities for each subject
    num_sims : int
            The number of draws from the distribution to be returned

    Returns
    -------
    prior_draws: np.array, shape (`num_sims`,)
    Indices into the `suspects` array for the generated suspects.
    """
    return np.random.choice(len(suspects), (num_sims,), p=probs)


def _draw_model(weapons, probs, s):
    # np.random.choice can't be used in vectorized form unless each sample has the same probability distribution.
    # As such, we make a function that calculates a single entry and use np.vectorize to combine the results
    return np.random.choice(len(weapons), p=probs[s,:])

draw_model=np.vectorize(_draw_model, excluded={0,1})

def draw_joint(weapons, suspects, prior_probs,likelihoods, num_sims):
    # s and w are arrays of indices, which have to be looked up in the suspects and weapons arrays respectively.
    s = draw_prior(suspects, prior_probs, num_sims)
    w = draw_model(weapons, likelihoods, s)
    # vectorized string concatenation: produces format {suspect}/{weapon} for each draw.
    return np.char.add(np.char.add(suspects[s], np.full(num_sims,"/")), weapons[w])

def joint_probability(scenario, suspects, weapons, prior_probs, likelihoods):
    """Returns the joint probability of a suspect-weapon pair."""
    suspect, weapon = scenario.split("/")
    suspect_idx=suspects.tolist().index(suspect)
    weapon_idx=weapons.tolist().index(weapon)
    return likelihoods[suspect_idx, weapon_idx] * prior_probs[suspect_idx]

def posterior_probability(scenario, suspects, weapons, prior_probs, likelihoods):
    suspect, weapon = scenario.split("/")
    suspect_idx=suspects.tolist().index(suspect)
    weapon_idx=weapons.tolist().index(weapon)
    # P(B)=sum[i]{ P(a_i|B)*P(a_i) }
    weapon_probability=(likelihoods[:,weapon_idx]*prior_probs).sum()
    return likelihoods[suspect_idx, weapon_idx] * prior_probs[suspect_idx]/weapon_probability

def simulator(weapons, suspects, prior_probs, likelihoods, num_sims = 100):
    """
    Simulates murder mystery scenario by drawing from the joint probability
    distribution of suspects and murder weapons.
    Returns an array of strings for each simulated scenario.

    Parameters
    ----------
    weapons : array-like, shape (M,)
            The array of weapons possibly used by the suspect.
    suspects : array-like, shape (N,)
            The array of suspects to draw from.
    prior_probs : array-like, shape (N,)
            The prior probabilities for each subject.
    likelihoods : array-like, shape (M,N)
            The likelihoods for each weapon being used by each suspect, rendered as a 2D array.
    num_sims : int
            The number of draws from the distribution to be returned.

    Returns
    -------
    joint_samples: np.array, shape (`num_sims`,)
    Array of strings of the format `{suspect}/{weapon}` for each sample.
    """
    return draw_joint(weapons, suspects, prior_probs, likelihoods, num_sims)

num_sims = 50_000

sims = simulator(weapons, suspects, prior_probs, likelihoods_arr, num_sims)

scenarios, counts = np.unique(sims, return_counts=True)

In [41]:
# Rendering scenario probabilities in a Pandas table
tbl = pd.DataFrame(columns = ["suspect/weapon", "Pr_empirical", "Pr_theory"])

# sorting table from least to most likely
for s, c in zip(np.array(scenarios)[np.argsort(counts)], np.sort(counts)):
    tbl.loc[len(tbl)] = [
        # scenario string
        s,
        # empirical probability
        c / num_sims,
        # prior probability for subject multiplied by likelihood function for weapon
        joint_probability(s,suspects,weapons,prior_probs,likelihoods_arr)
    ]
# in keeping with the 4-digit rounding from the in-class example (I think?)
pd.options.display.float_format = '{:.4f}'.format

# Adding a column to show the discrepancy between empirical and theoretical probabilities
tbl["Pr_empirical-Pr_theory"] = (tbl["Pr_empirical"] - tbl["Pr_theory"])

tbl

,suspect/weapon,Pr_empirical,Pr_theory,Pr_empirical-Pr_theory
0,engineer/raygun,0.0005,0.0006,-0.0001
1,engineer/parasite-beetle,0.0006,0.0006,-0.0000
2,captain/teleporter-hack,0.0009,0.0012,-0.0003
3,engineer/artifact,0.0011,0.0010,0.0001
4,engineer/brain-hack,0.0012,0.0010,0.0002
5,captain/brain-hack,0.0013,0.0012,0.0001
6,engineer/teleporter-hack,0.0049,0.0048,0.0001
7,pilot/teleporter-hack,0.0051,0.0056,-0.0005
8,pilot/parasite-beetle,0.0054,0.0056,-0.0002
9,prisoner/raygun,0.0081,0.0076,0.0005


To figure out how many samples it takes for the empirical probabilities to match the theoretical values, we can calculate the standard deviation of the distribution for each scenario. Because each run of the simulation is independent and has the same probability of producing a given scenario, the number of occurrences of a scenario follows a binomial distribution $n\sim\operatorname{Bin}(N_{sim},p)$, with $N_{sim}$ being the number of draws and $p$ being the joint probability of the scenario. This distribution has expectation value $\mathbb{E}[n]=pN_{sim}$ and standard deviation $\operatorname{Var}[n]={N_{sim}p(1-p)}$. As such, the distribution of $p_{empirical}=n/N_{sim}$ has expectation value $\mathbb{E}[p_{empirical}]=p$, and its standard deviation is $\sigma_{p_{empirical}}=\sqrt{\operatorname{Var}[n]/N_{sim}}=\sqrt{p(1-p)/N_{sim}}$. Note that if no probability is greater than 0.5 (which none of the joint probabilities are), increasing $p$ also increases the standard deviation. Thus, we should expect the most likely scenarios have empirical probabilities that deviate the most from the theoretical value, which does appear to be the case in the above table.

To be confident that our empirical probabilities are 'almost indistinguishable' from the true probabilities to 4 decimal places, we need the standard deviation to be small enough that 'almost all' points fall within $(10^{-4})/2$ of the mean (so that it rounds to the same value). If we decide that 99.7% is a reasonable definition for 'almost all', the 68-95-99.7 rule suggests we need $3\sigma\leq 5\cdot 10^{-5}$. Because higher-probability events have higher standard deviation, we should ensure this is true for the most likely outcome (`prisoner/teleporter-hack`, probability = 0.1330):

$$
\begin{gather*}
3\sigma=3\sqrt{\frac{(0.1330)(1-0.1330)}{N_{sim}}}\leq 5\cdot 10^{-5} \to \frac{(0.1330)(1-0.1330)}{N_{sim}}\leq\left(\frac{5\cdot 10^{-5}}{3}\right)^2\\
N_{sim}\geq\frac{(0.1330)(1-0.1330)}{\left((5\cdot 10^{-5})/{3}\right)^2}\to N_{sim}\geq 4.151\cdot 10^8
\end{gather*}
$$

Therefore, if we really wanted to ensure that the empirical probabilities match the theoretical values all the way to 4 places, we would need somewhere around 400 million samples.